In [57]:
import os.path
import sys

import keras.metrics
from keras import callbacks, models, metrics
from datahandler.constants import all_features, data_version, acc_features, tensorboard_dir, location_labels, mag_features
from datahandler.data_preprocessing import get_train_test_data, load_data_v3
from models.log_writer import LogWriter
from model.lstm import make_lstm_model_v1
from model.transformer import make_transformer_model_v1, make_transformer_model_v2
from model.cnn import make_cnn_model_v1, make_cnn_model_v2
import matplotlib.pyplot as plt
from utils import print_line_divider
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from datahandler.data_preprocessing_from_jupyter import load_train_test_data_raw_normalized,load_train_test_data_added_features_normalized, load_train_test_data_added_features_pca
from audio.audio import play_training_is_complete
from sklearn.metrics import classification_report
import time

In [21]:
# DATA Loader
print_line_divider()
print("Preparing data...")
# x_train, y_train, x_test, y_test = get_train_test_data(supported_features, window_time_in_seconds, window_size)
# x_train, y_train, x_test, y_test = load_data_v3(
#     features = supported_features,
#     window_time_in_seconds = window_time_in_seconds,
#     window_size = window_size
# )
x_train, y_train, x_test, y_test = load_train_test_data_raw_normalized(selected_features=all_features)
print("Train data shape: " + str(x_train.shape) + " | Train label shape: " + str(y_train.shape))
print("Test data shape: " + str(x_test.shape) + " | Test label shape: " + str(y_test.shape))
print_line_divider()


****************************************************
Preparing data...
Loading from file: /Users/duc.letran/Desktop/FINAL PROJECT/context_transformer/data/v4/mix/mm5_datacollection.csv (1/25)



KeyboardInterrupt



In [87]:
# LOGGING CONFIGURATION
enabled_log = True
enabled_tensor_board = True
log_writer = LogWriter(enabled_log)

In [88]:
# TRAINING CONFIGURATION
print("STARTING THE TRAINING PROCESS")
SAVED_BEST_MODEL = "model/best_model.h5"
window_time_in_seconds = 2
window_size = 40
epochs = 1000
batch_size = 32
validation_split = 15 / 85
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.01,
    name="Adam",
)
loss_function = "sparse_categorical_crossentropy"
supported_features = all_features

STARTING THE TRAINING PROCESS


In [89]:
log_writer.write("Configuration", line_divider=True)
log_writer.write(
    f"""Window size: {window_size}
Window time in second: {window_time_in_seconds}
Maximum number of epochs: {epochs}
Batch size: {batch_size}
Validation split: {validation_split}
Optimizer: {optimizer}
Loss function: {loss_function}"""
)
log_writer.write("Data", line_divider=True)
log_writer.write("Data loaded from version " + data_version)
log_writer.write(
    f"""Data training shape: ${x_train.shape}
Data testing shape: ${x_test.shape}"""
)

In [90]:
def check_distribution(labels_list, set_name):
    label_count_dict = {}
    for label_ind in labels_list:
        if label_ind in label_count_dict.keys():
            label_count_dict[label_ind] += 1
        else:
            label_count_dict[label_ind] = 1

    print_line_divider()
    print("Distribution for set " + set_name + "\n")
    for key in label_count_dict:
        key = int(key)
        print(location_labels[key] + " count is " + str(label_count_dict[key]))


check_distribution(y_train, "Training set")
check_distribution(y_test, "Testing set")

****************************************************
Distribution for set Training set

holdinginhand count is 1285
insidethepantpocket count is 1288
calling count is 1303
beingusedinhand count is 1302
insidethebag count is 1279
****************************************************
Distribution for set Testing set

holdinginhand count is 215
insidethepantpocket count is 215
calling count is 198
beingusedinhand count is 199
insidethebag count is 221


In [91]:
if len(x_train.shape) == 2:
    input_shape = (x_train.shape[1], 1)
else:
    input_shape = x_train.shape[1:]
print("Input shape: " + str(input_shape))

Input shape: (40, 9)


In [92]:
# SETTING UP THE MODEL
# model_name, model = make_cnn_model_v1(input_shape=input_shape)
# model_name, model = make_lstm_model_v1(input_shape=input_shape)
model_name, model = make_transformer_model_v1(
    input_shape=input_shape,
    head_size=64,
    num_heads=6,
    ff_dim=4,
    num_transformer_blocks=2,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)
# model_name, model = make_transformer_model_v2(input_shape=input_shape)
print("Model Summary:")
stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
short_model_summary = "\n".join(stringlist)
print(short_model_summary)
print(print_line_divider())

Model Summary:
Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 40, 9)]      0           []                               
                                                                                                  
 layer_normalization_12 (LayerN  (None, 40, 9)       18          ['input_11[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_6 (MultiH  (None, 40, 9)       14985       ['layer_normalization_12[0][0]', 
 eadAttention)                                                    'layer_normalization_12[0][0]'] 
                                                                            

In [93]:
# LOGGING THE MODEL
log_writer.write("Model", line_divider=True)
log_writer.write("Model name: " + model_name)
log_writer.write(short_model_summary)

In [94]:
# MODELS CALLBACK AND SETUP
callback_list = [
    callbacks.ModelCheckpoint(SAVED_BEST_MODEL, save_best_only=True, monitor="val_loss"),
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=50, min_lr=0.0001),
    callbacks.EarlyStopping(monitor="val_loss", patience=200, verbose=1)
]
if enabled_tensor_board:
    callback_list.append(callbacks.TensorBoard(log_dir=tensorboard_dir, histogram_freq=1))
if log_writer.enabled:
    callback_list.append(
        callbacks.ModelCheckpoint(log_writer.base_folder + "/model.h5", save_best_only=True, monitor="val_loss")
    )
model.compile(
    optimizer=optimizer,
    loss=loss_function,
    metrics=["sparse_categorical_accuracy"],
)

In [ ]:
# TRAINING
print_line_divider()
print("Starting to train...")

start_time = time.time()
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callback_list,
    validation_split=validation_split,
    verbose=1,
    shuffle=True
)
end_time = time.time()
training_time = end_time - start_time
print("Total training time in seconds: " + str(training_time))
print("Highest validation accuracy: ", max(history.history['val_sparse_categorical_accuracy']))
log_writer.write("Result", line_divider=True)
log_writer.write("Training time: " + str(end_time - start_time) + " seconds.")
log_writer.write("Highest validation accuracy: " + str(max(history.history['val_sparse_categorical_accuracy'])))
play_training_is_complete()

****************************************************
Starting to train...
Epoch 1/1000
 24/167 [===>..........................] - ETA: 11s - loss: 1.5408 - sparse_categorical_accuracy: 0.3203

In [ ]:
if log_writer.enabled:
    model.save(log_writer.base_folder + "/model_last.h5")

In [ ]:
# PLOTTING
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + model_name + " :" + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
if log_writer.enabled:
    plt.savefig(os.path.join(log_writer.base_folder, "Validation progress.png"))
plt.show()
plt.close()

In [ ]:
# EVALUATION
model = models.load_model(SAVED_BEST_MODEL)
test_loss, test_acc = model.evaluate(x_test, y_test)
print_line_divider()
print("Test accuracy", test_acc)
print("Test loss", test_loss)

# Accuracy based on different labels
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
metrics_report = classification_report(y_test, y_pred)
print("Metrics report: ")
print(metrics_report)

con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=4)
con_mat_df = pd.DataFrame(con_mat_norm, index=location_labels, columns=location_labels)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
if log_writer.enabled:
    plt.savefig(os.path.join(log_writer.base_folder, "Accuracy.png"))
plt.show()
plt.close()


In [ ]:
log_writer.write("Test evaluation", line_divider=True)
log_writer.write("Test accuracy: " + str(test_acc))
log_writer.write("Test loss: " + str(test_loss))
log_writer.write("Metric report: ")
log_writer.write(metrics_report)
log_writer.close()